### Train with NeMo

[Neural Modules (NeMo)](https://nvidia.github.io/NeMo/index.html) is a framework-agnostic toolkit for building AI applications. It currently supports the PyTorch framework.

Using NeMo to train a PyTorch model is simple. In this notebook, we will demonstrate how to use NeMo to train the Asian Barrier Option pricing model

In [ ]:
!pip install Cython
!pip install nemo_toolkit[all]==1.0.0rc1

     |████████████████████████████████| 747kB 17.3MB/s 
     |████████████████████████████████| 7.0MB 59.1MB/s 
     |████████████████████████████████| 1.2MB 56.0MB/s 
     |████████████████████████████████| 2.1MB 50.4MB/s 
     |████████████████████████████████| 12.2MB 227kB/s 
     |████████████████████████████████| 686kB 54.1MB/s 
     |████████████████████████████████| 133kB 62.7MB/s 
     |████████████████████████████████| 7.6MB 51.9MB/s 
     |████████████████████████████████| 12.8MB 254kB/s 
     |████████████████████████████████| 768kB 51.8MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 3.1MB 50.4MB/s 
     |████████████████████████████████| 2.1MB 44.8MB/s 
     |████████████████████████████████| 10.3MB 53.0MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 
     |████████████████████████████████| 1.7MB 48.

Defining the trainable module is similar to defining a PyTorch module but it defines the input and output ports:-

In [ ]:
%%writefile nemo_model.py
import torch.nn as nn
import torch.nn.functional as F
import torch
from nemo.core.neural_types import BatchTag, ChannelTag, NeuralType, AxisType
import nemo

class Net(nemo.backends.pytorch.nm.TrainableNM):
#class Net(nn.Module):
    @staticmethod
    def create_ports():
        input_ports = {"x": NeuralType({0: AxisType(BatchTag),
                                        1: AxisType(ChannelTag, 6)})}
        output_ports = {"y_pred": NeuralType({0: AxisType(BatchTag),
                                              1: AxisType(ChannelTag, 1)})}
        return input_ports, output_ports

    def __init__(self, hidden=512, **kwargs):
        super(Net, self).__init__(**kwargs)
        self.fc1 = nn.Linear(6, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, 1)
        self.register_buffer('norm',
                             torch.tensor([200.0,
                                           198.0,
                                           200.0,
                                           0.4,
                                           0.2,
                                           0.2]))

    def forward(self, x):
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        return self.fc6(x)

Overwriting nemo_model.py


The NeMo DataLayer module is wrapped around the normal PyTorch Dataset:-

In [ ]:
%%writefile nemo_datalayer.py
import torch
import nemo
from nemo.core.neural_types import BatchTag, ChannelTag, NeuralType, AxisType


class OptionDataSet(torch.utils.data.Dataset):
    def __init__(self, filename, rank=0, world_size=5):
        tensor = torch.load(filename)
        self.tensor = (tensor[0], tensor[1])
        self.length = len(self.tensor[0]) // world_size
        self.world_size = world_size
        self.rank = rank

    def __getitem__(self, index):
        index = index * self.world_size + self.rank

        return self.tensor[0][index], self.tensor[1][index]

    def __len__(self):
        return self.length

class OptionDataLayer(nemo.backends.pytorch.nm.DataLayerNM):
    @staticmethod
    def create_ports():
        # Note: we define the size of the height and width of our output
        # tensors, and thus require a size parameter.
        input_ports = {}
        output_ports = {
            "x": NeuralType({0: AxisType(BatchTag),
                                 1: AxisType(ChannelTag, 6)}),
            "ground": NeuralType({0: AxisType(BatchTag)})
        }
        return input_ports, output_ports

    def __init__(self, filename, rank=0, world_size=5, **kwargs):
        super().__init__(**kwargs)
        self._dataset = OptionDataSet(filename, rank, world_size)

    def __len__(self):
        return len(self._dataset)

    @property
    def dataset(self):
        return self._dataset

    @property
    def data_iterator(self):
        return None

Overwriting nemo_datalayer.py


We define the Loss Neural Module as following, which wraps around the PyTorch MSELoss with added input and output types:-

In [ ]:
%%writefile nemo_losslayer.py
import nemo
from nemo.core.neural_types import BatchTag, ChannelTag, NeuralType, AxisType
import torch

class MSELoss(nemo.backends.pytorch.nm.LossNM):
    @staticmethod
    def create_ports():
        input_ports = {"y_pred": NeuralType({0: AxisType(BatchTag),
                                             1: AxisType(ChannelTag, 1)}),
                       "ground": NeuralType({0: AxisType(BatchTag)})}
        output_ports = {"loss": NeuralType(None)}
        return input_ports, output_ports

    def __init__(self, **kwargs):
        # Neural Module API specific
        super().__init__(**kwargs)
        # End of Neural Module API specific
        self._loss = torch.nn.MSELoss()

    # You need to implement this function
    def _loss_function(self, **kwargs):
        v = self._loss(kwargs['y_pred'][:,0], kwargs['ground'])
        return v

Overwriting nemo_losslayer.py


In [ ]:
!pip install nemo-asr # installs NeMo ASR collection
!pip install nemo-nlp # installs NeMo NLP collection
!pip install nemo-tts # installs NeMo TTS collection

     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 102kB 6.5MB/s 
     |████████████████████████████████| 112kB 22.4MB/s 
     |████████████████████████████████| 235kB 17.5MB/s 
     |████████████████████████████████| 184kB 51.2MB/s 
ERROR: nemo 3.9.2 has requirement python-dateutil==2.8.1, but you'll have python-dateutil 2.8.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: botocore 1.20.60 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1


To use Neural Modules, we need to following 3 steps:-

1. Creation of NeuralModuleFactory and necessary NeuralModule
2. Defining a Directed Acyclic Graph (DAG) of NeuralModule
3. Call to “action” such as train

In [ ]:
import nemo
from nemo.core import DeviceType
from nemo_model import Net
from nemo_datalayer import OptionDataLayer
from nemo_losslayer import MSELoss
nf = nemo.core.NeuralModuleFactory()
# nf = nemo.core.NeuralModuleFactory()
dl= OptionDataLayer('trn.pth', 0, 1, batch_size=32)

# instantiate necessary neural modules
fx = Net(hidden=512).cuda() #, placement=DeviceType.GPU)
loss = MSELoss()

# describe activation's flow
x, y = dl()
p = fx(x=x)
lss = loss(y_pred=p, ground=y)

# SimpleLossLoggerCallback will print loss values to console.
callback = nemo.core.SimpleLossLoggerCallback(
    tensors=[lss],
    print_func=lambda x: print(f'Train Loss: {str(x[0].item())}'))

# Invoke "train" action
nf.train([lss], callbacks=[callback],
         optimization_params={"num_epochs": 20, "lr": 0.0003},
         optimizer="adam")

ImportError: ignored

NVIDIA Volta and Turing GPUs have Tensor Cores which can do fast matrix multiplications with values in float16 format. To enable mixed-precision in NeMo all you need to do is to set the optimization_level parameter of nemo.core.NeuralModuleFactory to nemo.core.Optimization.mxprO1. For example:

In [ ]:
nf = nemo.core.NeuralModuleFactory(optimization_level=nemo.core.Optimization.mxprO1)

For multi-GPU training, follow two steps in NeMo:
1. Set placement to nemo.core.DeviceType.AllGpu in NeuralModuleFactory
2. Add the ‘local_rank’ argument to your script and do not set it yourself: parser.add_argument(“–local_rank”, default=None, type=int)

In [ ]:
%%writefile nemo_dis_train.py
import nemo
from nemo.core import DeviceType
from nemo_model import Net
from nemo_datalayer import OptionDataLayer
from nemo_losslayer import MSELoss
import argparse
import os
parser = argparse.ArgumentParser(description='ResNet50 on ImageNet')
parser.add_argument("--local_rank", default=None, type=int)

args = parser.parse_args()

if args.local_rank is not None:
    device = nemo.core.DeviceType.AllGpu
else:
    device = nemo.core.DeviceType.GPU
    
world_size = int(os.environ['WORLD_SIZE'])

nf = nemo.core.NeuralModuleFactory(backend=nemo.core.Backend.PyTorch,
    local_rank=args.local_rank,
    placement=device,                               
    optimization_level=nemo.core.Optimization.mxprO1)
# nf = nemo.core.NeuralModuleFactory()
dl= OptionDataLayer('trn.pth', args.local_rank, world_size, batch_size=32)

# instantiate necessary neural modules
# RealFunctionDataLayer defaults to f=torch.sin, sampling from x=[-4, 4]
fx = Net(hidden=512).cuda() #, placement=DeviceType.GPU)
loss = MSELoss()

# describe activation's flow
x, y = dl()
p = fx(x=x)
lss = loss(y_pred=p, ground=y)

# SimpleLossLoggerCallback will print loss values to console.
callback = nemo.core.SimpleLossLoggerCallback(
    tensors=[lss],
    print_func=lambda x: print(f'Train Loss: {str(x[0].item())}'))

# Invoke "train" action
nf.train([lss], callbacks=[callback],
         optimization_params={"num_epochs": 20, "lr": 0.0003},
         optimizer="adam")

Overwriting nemo_dis_train.py


In [ ]:
!python -m torch.distributed.launch --nproc_per_node=4 nemo_dis_train.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Doing distributed training
Doing distributed training
Doing distributed training
2019-11-18 21:59:47,135 - WARNING - Data Layer does not have any weights to return. This get_weights call returns None.
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True

The [callback API](https://nvidia.github.io/NeMo/tutorials/callbacks.html) makes setting up check points and evaluating the validation dataset easy. Interested readers please check the document for details.